In [125]:
import csv
import datetime

In [194]:
def row_filler(hour, minute, seconds, poles, freq, res, curr, volt, slip):
    ns = round((120 * freq / poles), 2)
    nr = round((ns - (ns * slip)), 2)
    power = round((3 * res * curr * curr * nr / slip / ns), 2)
    torque = round(((3 * volt * volt * slip) / ((0.1047197 * ns) * res)),2)
    temp = 23 + (power/1000)
    return ns, nr, temp, power, torque

In [203]:
# Overwrites existing file

dt = datetime.datetime(2019,1,4, tzinfo=datetime.timezone.utc)
start = (dt - datetime.datetime(1970, 1, 1, tzinfo=datetime.timezone.utc)).total_seconds()
print(datetime.datetime.fromtimestamp(start, tz=datetime.timezone.utc).isoformat()[:-6])

filename = 'datalog_{}.csv'.format(dt.isoformat()[:10])

header = ['unit_id',
          'datetime',
          'Poles',
          'Frequency (Hz)', 
          'Resistance (Ohm)',
          'Current (A)',
          'Voltage (V)',
          'Slip',
          'ns (rpm)', 
          'nr (rpm)',
          'Temperature (C)',
          'Power (W)',
          'Torque (N-m)']

with open(filename, 'w', newline='') as csvfile:
    mywriter = csv.writer(csvfile, delimiter=',')
    mywriter.writerow(header)

2019-01-04T00:00:00


In [204]:
unit_id='teco_lab_71B'
diffsecs = 0
temp = 23
freq = 40
poles = 4
res = .25
curr = 100
volt = 200
slip = 0.25

with open(filename, 'a', newline='') as csvfile:
    mywriter = csv.writer(csvfile, delimiter=',')

    for hour in range(24):
        if hour%2 == 0:
            freq += 5
            if hour%4 == 0:
                curr += 20
        elif hour == 11:
            freq = 50
            curr = 100
        for minute in range(60):
            if minute == 0:
                volt = 200
            elif minute%10 == 0:
                volt += 20
            for seconds in range(60):
                dt = datetime.datetime.fromtimestamp(start+diffsecs, tz=datetime.timezone.utc).isoformat()[:-6]
                ns, nr, temp, power, torque = row_filler(hour, minute, seconds, poles, freq, res, curr, volt, slip)
                if minute <= 30:
                    temp += (minute*60 + seconds)/1800 * 20
                else:
                    temp += 20
                mywriter.writerow([unit_id, 
                       dt,
                      poles,
                      freq, 
                      res,
                      curr,
                      volt,
                      slip,
                      ns, 
                      nr,
                      round(temp, 2),
                      power,
                      torque])
                diffsecs += 1

In [24]:
# with open(filename, 'a', newline='') as csvfile:
#     mywriter = csv.writer(csvfile, delimiter=',')
#     mywriter.writerow(row_filler())